In [5]:
# Data Manipulation and Analysis
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Statistical Modeling and Machine Learning
from sklearn import model_selection, metrics, linear_model, ensemble
import statsmodels.api as sm
from scipy import stats
#excel files
import openpyxl


In [6]:
#Viewing Options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 308)
pd.set_option('display.width', None)

In [8]:
OG_fangraphs = pd.read_csv("fangraphs_season_level.csv")
OG_savant = pd.read_csv("savant_pitch_level.csv")

In [11]:
OG_savant['game_year']

0          2021
1          2021
2          2021
3          2021
4          2021
           ... 
2136332    2023
2136333    2023
2136334    2023
2136335    2023
2136336    2023
Name: game_year, Length: 2136337, dtype: int64

In [15]:
def filterBy(dataframe, minimumPitchesNeeded = 0, year = 0):
    if year != 0:
        dataframe = dataframe[dataframe['game_year'] == year]
        
    player_counts = dataframe['player_name'].value_counts()
    
    # Filter the dataframe to include only players with at least x appearances
    filtered_dataframe = dataframe[dataframe['player_name'].isin(player_counts[player_counts >= minimumPitchesNeeded].index)]
    
    return filtered_dataframe


filteredSavant = filterBy(OG_savant, 0, 2023)

In [19]:
len(filteredSavant['player_name'].unique())

479

In [25]:
OG_fangraphs.columns

Index(['PlayerId', 'MLBAMID', 'Name', 'NameASCII', 'Throws', 'Season', 'Age',
       'Team', 'Role', 'G',
       ...
       'Pit_plus_FC', 'Stf_plus_FS', 'Loc_plus_FS', 'Pit_plus_FS',
       'Stf_plus_FO', 'Loc_plus_FO', 'Pit_plus_FO', 'Stuff_plus',
       'Location_plus', 'Pitching_plus'],
      dtype='object', length=308)

In [9]:
import pandas as pd

#Finds the average of the "Two halves" for any column

def calculate_average_dif(df, player_name, column):
    #Filter the rows
    player_rows = df[df['player_name'].str.contains(player_name, case=False, na=False)]

    #Sort the stuff
    sorted_player_rows = player_rows.sort_values(by='pitch_number_appearance')

    # Split the sorted rows 
    first_half_player = sorted_player_rows.iloc[:len(sorted_player_rows) // 2]
    second_half_player = sorted_player_rows.iloc[len(sorted_player_rows) // 2:]

    # Find the averages
    average_delta_run_exp_first_half = first_half_player[column].mean()
    average_delta_run_exp_second_half = second_half_player[column].mean()

    return average_delta_run_exp_first_half, average_delta_run_exp_second_half


#Finds
def calculateAllHalvesDifference(df,column):
    unique_player_names = OG_savant['player_name'].unique()
    players = []
    avg_first_half = []
    avg_second_half = []
    for player_name in unique_player_names:
        average_first_half, average_second_half = calculate_average_dif(OG_savant, player_name, column)
        players.append(player_name)
        avg_first_half.append(average_first_half)
        avg_second_half.append(average_second_half)
    difference = [second - first for first, second in zip(avg_first_half, avg_second_half)]
    result_df = pd.DataFrame({
    'Player': players,
    'Average_First half: ' + column: avg_first_half,
    'Average_Second half: ' + column: avg_second_half,
    'Difference: ' + column: difference
    })
    return result_df


calculated_df = calculateAllHalvesDifference(OG_savant,"delta_run_exp")

KeyboardInterrupt: 